In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.metrics import accuracy_score
from pprint import pprint

In [2]:
class Node:
    def __init__(self, feature=None, threshold=1e-4, left=None, right=None, value=None):
        self.left = left
        self.right = right
        self.value = value
        self.feature = feature
        self.threshold = threshold

In [3]:
class DecisionTreeClassifier:
    def __init__(self, max_depth=5):
        self.max_depth = max_depth

    def fit(self, X, y):
        self.n_features_ = X.shape[1]
        self.tree_ = self.splitting(X, y)
        return self.tree_

    def predict(self, X):
        def traverse(node):
            if node.left is None:
                return node.value
            if x[node.feature] < node.threshold:
                return traverse(node.left)
            else:
                return traverse(node.right)

        predictions = []
        for x in X:
            predictions.append(traverse(self.tree_))
        return predictions

    def splitting(self, X, y, depth=0):
        n_labels = len(np.unique(y))
        no_of_samples = X.shape[0]
        no_of_features = X.shape[1]

        if n_labels == 1 :
            return Node(value=Counter(y).most_common(1)[0][0])
        elif no_of_samples < 2:
            return Node(value=Counter(y).most_common(1)[0][0])
        elif depth == self.max_depth:
            return Node(value=Counter(y).most_common(1)[0][0])

        best_criteria = np.inf
        best_feature = None
        best_threshold = None
        for feature in range(no_of_features):
            thresholds = np.unique(X[:, feature])
            criteria = np.array([len(y[X[:, feature] < threshold]) * self._entropy(y[X[:, feature] < threshold]) +
                                len(y[X[:, feature] >= threshold]) * self._entropy(y[X[:, feature] >= threshold])
                                for threshold in thresholds])
            best_threshold_idx = np.argmin(criteria)
            if criteria[best_threshold_idx] < best_criteria:
                best_criteria = criteria[best_threshold_idx]
                best_feature = feature
                best_threshold = thresholds[best_threshold_idx]

        mask = X[:, best_feature] < best_threshold
        X_left, X_right = np.split(X, [mask.sum()])
        y_left, y_right = np.split(y, [mask.sum()])

        left = self.splitting(X_left, y_left, depth + 1)
        right = self.splitting(X_right, y_right, depth + 1)

        return Node(feature=best_feature, threshold=best_threshold, left=left, right=right)

    def _entropy(self, y):
        n = len(y)
        unique_labels = np.unique(y)
        entropy = 0.0
        for label in unique_labels:
            p = np.sum(y == label) / n
            entropy -= p * np.log2(p)
        return entropy

    def accuracy(self, predicted_y, actual_y):
        return accuracy_score(predicted_y, actual_y)


In [4]:
dataset = load_iris()
X = dataset.data
y = dataset.target

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [6]:
model = DecisionTreeClassifier(max_depth=5)
tree = model.fit(X_train, y_train)

In [7]:
y_pred = model.predict(X_test)

In [8]:
print("Accuracy:")
print(model.accuracy(y_pred, y_test))

Accuracy:
0.9333333333333333
